# Refining Fandango Constraints using Avicenna

In our previous exploration, we saw how **FandangoLearn** could derive constraints that separate failing inputs from passing inputs for a particular program behavior (in this case, a simple calculator-like subject using functions like `sqrt`, `sin`, `cos`, and `tan`). However, what we learned was only a first draft. The initial constraints are often too crude, missing nuances that accurately capture the essence of failing conditions.

**Why do we need refinement?** Imagine you’re trying to understand why certain inputs cause failures. You start with a set of initial inputs, such as sqrt(-900) or sqrt(-10). However, in the previous chapter, we have shown that based on a set of initial inputs, we are not able to learn perfect constraints. The constraint `str(<function>) == 'sqrt' and int(<number>) <= -10` fails to capture inputs like `sqt(-4)`. Thus, we need to refine them!

A refinement loop allows us to escape these early approximations. It works like this:

1. **Initial Learning:** We give **FandangoLearn** a set of known inputs, both failing and passing.
2. **Initial Constraints:** FandangoLearn tries to find patterns that distinguish these failing inputs from passing ones. Based on a limited input set, it might find something too restrictive, like a rule that only matches failures when `int(<number>) <= -10`.
3. **Challenge the Constraints:** We generate new inputs that are not yet distinguished by these constraints. This puts pressure on the current constraints to “fail,” exposing their weaknesses. For instance, if we test `sqrt(-6)`, we see it fails too, even though the constraint said `<= -10`.
4. **Refine:** Incorporate these new, challenging inputs back into the learning process, allowing **FandangoLearn** to home in on more accurate constraints. With enough rounds, we transition from rough guesses to finer, more representative constraints.

In essence, the refinement loop is a conversation between the learner and the inputs: the constraints try to explain failures, while new inputs (often automatically generated) poke holes in that explanation. Each round makes the constraints more accurate, relevant, and minimal.

## Motivation

In [1]:
import random
random.seed(3)

Let’s walk through an example to illustrate this process. We start with some initial constraints learned from a small set of inputs, show their limitations, then refine them by generating new inputs and guiding **FandangoLearn** to more precise constraints.

In [2]:
from fandangoLearner.interface.fandango import parse_file

from fandango.language.parse import Grammar
from fandangoLearner.data.input import FandangoInput

grammar_file = "calculator.fan"
grammar, _ = parse_file(grammar_file)
assert isinstance(grammar, Grammar), "Grammar is not loaded correctly"

# Our initial input set: a mix of passing and failing cases
initial_inputs = {
    ("sqrt(-900)", True),
    ("sqrt(-10)", True),
    ("sqrt(0)", False),
    ("sin(-900)", False),
    ("sqrt(2)", False),
    ("cos(10)", False),
}

# Parse and prepare the initial inputs
for inp, _ in initial_inputs:
    tree = grammar.parse(inp)
    assert tree is not None, f"Failed to parse {inp}"

initial_inputs = {FandangoInput.from_str(grammar, inp, oracle) for inp, oracle in initial_inputs}

At this stage, we apply **FandangoLearn** to these initial inputs. We’ll see it produce some candidate constraints. These constraints try to mark the difference between failing and passing, but they might overreach or miss edge cases.

In [3]:
from fandangoLearner.learner import FandangoLearner
from fandangoLearner.learner import NonTerminal
relevant_non_terminals = {
    NonTerminal("<number>"),
    NonTerminal("<maybeminus>"),
    NonTerminal("<function>"),
}

learner = FandangoLearner(grammar)
learned_constraints = learner.learn_constraints(initial_inputs, relevant_non_terminals)

fandango-learner:INFO: Instantiated patterns: 18
fandango-learner:INFO: Filtered positive inputs for learning: 2
fandango-learner:INFO: Found 12 valid conjunctions


We can review the best candidate constraints. They might look clever, but they could be overspecialized or simply too long-winded, pinpointing details that do not generalize (like focusing on “length of a string” or `<= -10` specifically).

In [4]:
for candidate in learner.get_best_candidates():
    print(candidate)

(int(<number>) <= -10 and str(<function>) == 'sqrt'), Precision: 1.0, Recall: 1.0 (based on 2 failing and 4 passing inputs)


You can see constraints that combine properties like int(<number>) <= -10.0 with checks like str(<function>) == 'sqrt'. It’s a start, but we need something more generally true. Right now, the constraints are based only on the inputs we gave. To test whether they truly reflect all failing conditions, we must challenge them with new inputs.

Next, we refine by adding new inputs that are generated to stretch or break these constraints. By doing this, we force the learner to confront gaps in its understanding.

In [5]:
import math
from debugging_framework.input.oracle import OracleResult

def calculator_oracle(inp: str) -> OracleResult:
    try:
        eval(
            str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
        )
    except ValueError:
        # If evaluation fails due to a domain error, we say it FAILS.
        return OracleResult.FAILING
    return OracleResult.PASSING

We use a search-based approach to create more inputs that match—or fail to be distinguished by—the current constraints. This helps us unearth hidden assumptions.

In [6]:
from fandango.evolution.algorithm import Fandango
from fandango.language.tree import DerivationTree

more_inputs = set(initial_inputs)
for candidate in learner.get_best_candidates():
    print(candidate)
    fandango = Fandango(grammar, [candidate.constraint], desired_solutions=10)
    solutions = fandango.evolve()
    for tree in solutions:
        more_inputs.add(FandangoInput(tree, calculator_oracle(str(tree))))

assert all(inp.oracle is not None for inp in more_inputs)
assert all(isinstance(inp.tree, DerivationTree) for inp in more_inputs)

(int(<number>) <= -10 and str(<function>) == 'sqrt'), Precision: 1.0, Recall: 1.0 (based on 2 failing and 4 passing inputs)


Inspecting these newly generated inputs, we see a broader range: negative numbers of various magnitudes, different functions, etc. This variety challenges the initial constraints.

In [7]:
for inp in more_inputs:
    print(inp.tree, candidate.constraint.check(inp))

sqrt(-1990) True
sqrt(-252000) True
sqrt(-900) True
sqrt(-70) True
sin(-900) False
sqrt(-191) True
sqrt(2) False
sqrt(-85792) True
sqrt(-31000) True
sqrt(-10) True
sqrt(0) False
sqrt(-304) True
sqrt(-203400) True
sqrt(-33) True
cos(10) False


We’ll next narrow down which parts of the grammar are truly relevant to failing conditions. Perhaps only certain non-terminals (like `<function>` and `<number>`) matter. This pruning leads to simpler, more coherent constraints.

In [8]:
from fandangoLearner.learner import NonTerminal
relevant_non_terminals = {
    NonTerminal("<number>"),
    NonTerminal("<maybeminus>"),
    NonTerminal("<function>"),
}

learner = FandangoLearner(grammar)
learned_constraints = learner.learn_constraints(more_inputs, relevant_non_terminals=relevant_non_terminals)

for candidate in learner.get_best_candidates():
    print(candidate.constraint)

fandango-learner:INFO: Instantiated patterns: 18
fandango-learner:INFO: Filtered positive inputs for learning: 5
fandango-learner:INFO: Found 25 valid conjunctions


(int(<number>) <= -10 and str(<function>) == 'sqrt')


Now the constraints are simpler, but still not perfect. They might say something like: int(\<number>) <= -10.0 and str(\<function>) == 'sqrt'. Sure, it’s closer, but we know that not only sqrt(-10), but also sqrt(-4) fails. The constraint is too strict in one sense (-10 is arbitrary) and not broad enough in another. To fix this, we can push even further.

At this point, we try something more radical: we negate part of the learned constraint to find inputs that fail the opposite condition. This is like saying, “If you think it has to be <= -10 for sqrt, what if we look for sqrt with numbers greater than -10?” This trick helps the learner step outside its current framing and find a more general rule.

By adding new inputs, we were able to reduce the number of constraints significantly.
Thus, we were able to exclude constraints that were not precise enough. However, you see that the constraint `int(<number>) <= -10 and str(<function>) == 'sqrt'` is still not perfect. Further refining this constraint by genereating more inputs with this constraint will not help, because we will only generate more inputs that will fulfil this constraint. However, we need to generate inputs that are not covered by this constraint. Inputs such as `sqrt(-2)` or `sqrt(-1)`. Thus we have to negate this constraint.

In [9]:
from fandango.constraints.base import *
from fandango.language.search import RuleSearch

class NegationConstraint(Constraint):
    """A simple constraint to represent logical negations."""
    def __init__(self, inner_constraint: Constraint, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.inner_constraint = inner_constraint

    def transform(self, transformer: "ConstraintTransformer") -> "Constraint":
        # Apply the transformer to the inner constraint.
        normalized_inner = self.inner_constraint.transform(transformer)
        return NegationConstraint(normalized_inner)

    def accept(self, visitor: "ConstraintVisitor"):
        #visitor.visit_expression_constraint(self)
        pass

    def fitness(
            self, tree: DerivationTree, scope: Optional[Dict[NonTerminal, DerivationTree]] = None
    ) -> ConstraintFitness:
        """
        Computes the fitness for the negation of the inner constraint.

        Negation logic:
        - If the inner constraint is fully satisfied, this constraint is fully unsatisfied.
        - If the inner constraint is fully unsatisfied, this constraint is fully satisfied.
        - Otherwise, the fitness is calculated as the negation of the inner fitness.
        """
        # Evaluate the fitness of the inner constraint
        inner_fitness = self.inner_constraint.fitness(tree, scope)

        # Negate the fitness results
        solved = 1 - inner_fitness.solved
        total = inner_fitness.total
        success = not inner_fitness.success

        return ConstraintFitness(
            solved=solved,
            total=total,
            success=success,
            # failing_trees=failing_trees,
        )

    def __repr__(self):
        return f"~({repr(self.inner_constraint)})"

In [10]:
from fandango.constraints.base import ConjunctionConstraint
negated_constraint = NegationConstraint(learned_constraints[0].constraint.constraints[0])
test_constraint = ConjunctionConstraint([negated_constraint, learned_constraints[0].constraint.constraints[1]])

In [11]:
test_constraint

(~(int(<number>) <= -10) and str(<function>) == 'sqrt')

In [12]:
fandango = Fandango(grammar, [test_constraint], desired_solutions=100)
solutions = fandango.evolve()

In [13]:
negated_inputs = {FandangoInput(tree, calculator_oracle(str(tree))) for tree in solutions}

new_failing_inputs = {inp for inp in negated_inputs if inp.oracle == OracleResult.FAILING}
for inp in new_failing_inputs:
    print(inp.tree)

sqrt(-4)
sqrt(-7)
sqrt(-3)
sqrt(-6)
sqrt(-9)


In [14]:
learner = FandangoLearner(grammar)
learned_constraints = learner.learn_constraints(
    more_inputs.union(new_failing_inputs),
    relevant_non_terminals=relevant_non_terminals
)
for candidate in learner.get_best_candidates():
    print(candidate.constraint)

fandango-learner:INFO: Instantiated patterns: 18
fandango-learner:INFO: Filtered positive inputs for learning: 5
fandango-learner:INFO: Found 25 valid conjunctions


(int(<number>) <= -3 and str(<function>) == 'sqrt')


We can see that we got much closer to the perfect constraint. However, we still have to refine the constraint further. We can do this by adding more inputs that are not yet distinguished by the learned constraints.

Now the constraint might say something like `int(<number>) <= -4.0 and str(<function>) == 'sqrt'`. We’re getting closer to a simpler, more accurate constraint that captures the failing behavior: as soon as we try sqrt with any number less than -4, it fails.

The refinement loop is about continually challenging and honing our understanding of what causes failures. By incrementally adding test inputs, flipping assumptions, and seeing which constraints still hold, we move from vague, overfitted rules toward constraints that truly represent the underlying pattern of failure.

This process can continue until we arrive at constraints that are minimal yet sufficient to explain the observed behavior. In our story, we’ve gone from messy, overly specific constraints to something much clearer and more robust.

## Automatically Refining Fandango Constraints

In [15]:
random.seed(3)
from fandangoLearner.refinement.core import FandangoRefinement,LoggerLevel

In [16]:
fandangoRE = FandangoRefinement(
    grammar,
    calculator_oracle,
    [str(inp) for inp in initial_inputs],
    relevant_non_terminals=relevant_non_terminals,
    logger_level=LoggerLevel.CRITICAL
)

In [17]:
candidates = fandangoRE.explain()

In [18]:
for candidate in candidates:
    print(candidate.constraint)

(int(<number>) <= -1 and str(<function>) == 'sqrt')
